<a href="https://colab.research.google.com/github/gwnwytt/GwenWyattDTSC3020Fall2025/blob/main/assignment5_gw0183.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [4]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [40]:
# Write your answer here
import re, csv
from pathlib import Path

rawfile = 'contacts_raw.txt'
path = Path(rawfile)
cleanfile = 'contacts_clean.csv'
seen = set()
clean_rows = []
email_pat = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

def normalize_phone(raw):
  digits = re.sub(r"\D", "", raw)
  if len(digits) >= 10:
    return digits[-10:]
  else:
    return ""

try:
  with open(rawfile) as f:
    lines = f.readlines()

  for line in lines:
    line = line.strip()

    emails = email_pat.findall(line)
    if not emails:
      continue
    email = emails[0]
    email_case = email.casefold()
    if email_case in seen:
      continue
    seen.add(email_case)

    find_phone = re.findall(r"[\d\+\-\(\)\s\.]+", line)
    phone = ''
    if find_phone:
      phone = normalize_phone(find_phone[-1])

    name = re.sub(r"<.*?>", "", line)
    name = name.replace(email, "").replace(",","").strip()

    clean_rows.append([name, email, phone])

  with open(cleanfile, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["name", "email", "phone"])
    writer.writerows(clean_rows)

except FileNotFoundError:
  print(f"File {rawfile!r} was not found")

## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [55]:
# Write your answer here
import re, io, csv, unittest

class TestDataCleaning(unittest.TestCase):

  def test_email_valid(self):
    valid_emails = [
        "gw3824u9@ex.edu",
        "nilsongoran@unt.com",
        "testuser894@blahblah.blah"
    ]
    for email in valid_emails:
      self.assertTrue(email_pat.fullmatch(email))

  def test_email_invalid(self):
    invalid_emails = [
        "2938rhfiwf[at]bob.com",
        '@email.com',
        'gwuser@unt'
    ]
    for email in invalid_emails:
      self.assertFalse(email_pat.fullmatch(email))

  def test_phone_valididation(self):
    self.assertEqual(normalize_phone("+1 123 456 7890"), "1234567890")
    self.assertEqual(normalize_phone("(123) 456 7890"), "1234567890")
    self.assertEqual(normalize_phone("123-456-7890"), "1234567890")
    self.assertEqual(normalize_phone("(123)-456-7890"), "1234567890")
    self.assertEqual(normalize_phone("+1 (123) 456 7890"), "1234567890")
    self.assertEqual(normalize_phone("(23) 456 7890"), "")
    self.assertEqual(normalize_phone("123-456-780"), "")

  def test_parsing_and_deduplication(self):
    fake_raw = """Gwen <gwen@gmail.com>, +1 (123)4567890
    fake perosn <noemail[at]noemail> , 4444444
    gwen Wyatt <Gwen@Gmail.com>, 123-456-7689
    No Name <userguy@ex.edu>, 111.111.1111"""

    fake_rawfile = io.StringIO(fake_raw)
    lines = fake_rawfile.readlines()

    seen = set()
    clean_rows = []

    for line in lines:
      line = line.strip()
      emails = email_pat.findall(line)
      if not emails:
        continue
      email = emails[0]
      email_case = email.casefold()
      if email_case in seen:
        continue
      seen.add(email_case)
      phone_match = re.findall(r"[\d\+\-\(\)\.]+", line)
      phone = normalize_phone(phone_match[-1]) if phone_match else ""
      name = re.sub(r"<.*?>", "", line)
      name = email_pat.sub("", name)
      name = re.sub(r"[\d\+\-\(\)\.]+", "", name)
      name = name.replace(email, "").replace(",", "").strip()
      clean_rows.append([name, email, phone])

    expected_data = [
      ["Gwen", "gwen@gmail.com", "1234567890"],
      ["No Name", "userguy@ex.edu", "1111111111"]
    ]

    self.assertEqual(clean_rows, expected_data)

if __name__ == "__main__":
  unittest.main(argv=[''], exit=False, verbosity=2)


test_email_invalid (__main__.TestDataCleaning.test_email_invalid) ... ok
test_email_valid (__main__.TestDataCleaning.test_email_valid) ... ok
test_parsing_and_deduplication (__main__.TestDataCleaning.test_parsing_and_deduplication) ... ok
test_phone_valididation (__main__.TestDataCleaning.test_phone_valididation) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.011s

OK


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
